# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_path = "../output_data/cities.csv"
cities_df = pd.read_csv(cities_path)
cities_df['City'] = cities_df.City.str.title()  
del cities_df['Unnamed: 0']
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Grindavik,63.84,-22.43,50.00,93,90,14.99,IS
1,Port-Gentil,-0.72,8.78,74.39,81,100,13.47,GA
2,Blythe,33.61,-114.60,118.99,9,4,3.00,US
3,Rikitea,-23.12,-134.97,72.43,80,10,13.31,PF
4,Souillac,-20.52,57.52,66.20,93,40,6.64,MU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = cities_df["Humidity"].astype(float)

In [13]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create Northern and Southern Hemisphere DataFrames
vacation_cities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) &
                                   (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
37,Carbonia,39.17,8.52,73.00,94,0,2.24,IT
74,Vilhena,-12.74,-60.15,76.03,29,0,3.80,BR
104,Los Llanos De Aridane,28.66,-17.92,71.60,78,0,5.82,ES
126,Arganil,40.22,-8.05,73.00,84,0,2.57,PT
170,Bardiyah,31.76,25.09,74.14,67,0,7.16,LY
263,Maneadero,31.72,-116.57,75.99,86,0,2.39,MX
280,Behshahr,36.69,53.55,72.37,67,0,2.68,IR
290,Maragogi,-9.01,-35.22,73.85,76,0,9.93,BR
313,Tabas,33.60,56.92,75.20,16,0,4.70,IR
328,Shahgarh,24.32,79.13,79.65,82,0,7.85,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
37,Carbonia,39.17,8.52,73.00,94,0,2.24,IT,
74,Vilhena,-12.74,-60.15,76.03,29,0,3.80,BR,
104,Los Llanos De Aridane,28.66,-17.92,71.60,78,0,5.82,ES,
126,Arganil,40.22,-8.05,73.00,84,0,2.57,PT,
170,Bardiyah,31.76,25.09,74.14,67,0,7.16,LY,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # Take lat, lng and cityname from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # Make API request
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name = response['results'][0]['name']
        print(f"Closest hotel in {city} is {hotel_name}")
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    # In case of IndexError print no nearest hotel found
    except IndexError:
        print(f"No hotel information found for {city}. Dropping from list...")
        hotel_df.drop(index=index, inplace=True)

Closest hotel in Carbonia is Lu 'Hotel
Closest hotel in Vilhena is Hotel Mirage
Closest hotel in Los Llanos De Aridane is Hotel Hacienda de Abajo
Closest hotel in Arganil is Hotel de Arganil
Closest hotel in Bardiyah is Hotel Burdy
No hotel information found for Maneadero. Dropping from list...
Closest hotel in Behshahr is Ashraf Hotel
Closest hotel in Maragogi is Salinas do Maragogi All Inclusive Resort
Closest hotel in Tabas is Amir hotel
Closest hotel in Shahgarh is Shubham Hotel
Closest hotel in Barcelos is Quinta de Santa Comba
Closest hotel in Ibra is Ibra Hotel
No hotel information found for Marzuq. Dropping from list...
Closest hotel in Caravelas is Pousada Encanto Abrolhos
No hotel information found for Najran. Dropping from list...
Closest hotel in Santa Cruz De La Palma is Hacienda San Jorge
No hotel information found for Kutum. Dropping from list...
Closest hotel in Korla is Kangcheng Jian Guo International Hotel
Closest hotel in Syracuse is Charme Hotel Henry's House / Hot

In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
37,Carbonia,39.17,8.52,73.00,94,0,2.24,IT,Lu 'Hotel
74,Vilhena,-12.74,-60.15,76.03,29,0,3.80,BR,Hotel Mirage
104,Los Llanos De Aridane,28.66,-17.92,71.60,78,0,5.82,ES,Hotel Hacienda de Abajo
126,Arganil,40.22,-8.05,73.00,84,0,2.57,PT,Hotel de Arganil
170,Bardiyah,31.76,25.09,74.14,67,0,7.16,LY,Hotel Burdy
280,Behshahr,36.69,53.55,72.37,67,0,2.68,IR,Ashraf Hotel
290,Maragogi,-9.01,-35.22,73.85,76,0,9.93,BR,Salinas do Maragogi All Inclusive Resort
313,Tabas,33.60,56.92,75.20,16,0,4.70,IR,Amir hotel
328,Shahgarh,24.32,79.13,79.65,82,0,7.85,IN,Shubham Hotel
382,Barcelos,41.54,-8.62,73.99,100,0,5.73,PT,Quinta de Santa Comba


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))